$\beta = -1$ means that $\beta$ is selected at random in each iteration.

In [6]:
from grasp_betas import read_results

save = True
names = [
    "ch150_100_50_0", # 0
    "pr439_293_146_0",
]
name = names[1]

betas = read_results(name)
betas

,p,alpha,0.0,0.5,1.0,-1,best,d_0.0,d_0.5,d_1.0,d_-1
0,7,2,3868,3868,3868,3868,3868,0.000000,0.000000,0.000000,0.000000
1,7,3,5433,5457,5433,5457,5433,0.000000,0.441745,0.000000,0.441745
2,14,2,2719,2719,2719,2719,2719,0.000000,0.000000,0.000000,0.000000
3,14,3,3530,3471,3734,3640,3471,1.699798,0.000000,7.577067,4.868914
4,29,2,2620,2620,2620,2620,2620,0.000000,0.000000,0.000000,0.000000
5,29,3,3222,3222,3222,3222,3222,0.000000,0.000000,0.000000,0.000000
